In [1]:
import pandas as pd
import requests
import json # Importa el módulo 'json'

In [ ]:
def obtener_datos(series_id, site_code):
    """
    Realiza una petición a la API y devuelve un DataFrame.
    """
    url = f"https://alerta.ina.gob.ar/pub/datos/datos&timeStart=2025-09-18&timeEnd=2025-09-20&seriesId={series_id}&siteCode={site_code}&varId=2&format=csv"
    
    try:
        # Lee el CSV directamente desde la URL.
        df = pd.read_csv(url, comment='#', sep=',', header=None)
        df.columns = ['obs_id', 'timestart', 'timeend', 'valor', 'timeupdate']
        
        # Agrega las columnas adicionales
        df['seriesId'] = series_id
        df['siteCode'] = site_code
        
        return df
    
    except Exception as e:
        print(f"Error al obtener datos para seriesId {series_id}: {e}")
        return pd.DataFrame() # Devuelve un DataFrame vacío en caso de error

# Leer el archivo JSON de forma síncrona con Python
try:
    with open("alturasHidrometricas.json", 'r', encoding='utf-8') as file:
        alturas_json = json.load(file)
except FileNotFoundError:
    print("Error: El archivo 'alturasHidrometricas.json' no se encontró.")
    alturas_json = []

lista_de_dataframes = []

for item in alturas_json:
    df_temporal = obtener_datos(item['seriesid'], item['sitecode'])
    lista_de_dataframes.append(df_temporal)

# Concatena todos los DataFrames en uno solo
df_final = pd.concat(lista_de_dataframes, ignore_index=True)

# Muestra las primeras 5 filas del DataFrame final
print(df_final.head())

# Opcional: Guarda el DataFrame final en un solo archivo CSV
df_final.to_csv("datos_finales.csv", index=False)

KeyboardInterrupt: 

In [2]:
df_nuevito = pd.read_csv("datos_finales.csv")
df_nuevito

,obs_id,timestart,timeend,valor,timeupdate,seriesId,siteCode
0,25671322243,2025-09-18 00:00:00,2025-09-18 00:00:00,0.66,2025-09-18 18:01:45.740123,8,8
1,25678429536,2025-09-19 00:00:00,2025-09-19 00:00:00,0.46,2025-09-19 12:01:56.589385,8,8
2,25671322611,2025-09-18 00:00:00,2025-09-18 00:00:00,8.60,2025-09-18 18:01:46.129692,9,9
3,25678429676,2025-09-19 00:00:00,2025-09-19 00:00:00,8.40,2025-09-19 12:01:56.830545,9,9
4,25671322635,2025-09-18 00:00:00,2025-09-18 00:00:00,9.20,2025-09-18 18:01:46.298099,10,10
...,...,...,...,...,...,...,...
10329,25675266304,2025-09-19 00:00:00,2025-09-19 00:00:00,1.48,2025-09-19 04:01:46.98231,37594,7392
10330,25676460271,2025-09-19 04:00:00,2025-09-19 04:00:00,1.45,2025-09-19 07:01:50.255738,37594,7392
10331,25677641323,2025-09-19 08:00:00,2025-09-19 08:00:00,1.42,2025-09-19 10:01:48.955388,37594,7392
10332,25679618659,2025-09-19 12:00:00,2025-09-19 12:00:00,1.40,2025-09-19 15:01:48.669583,37594,7392


In [3]:
df = df_nuevito.drop(['timestart', 'timeupdate', 'obs_id'], axis=1)

In [4]:
#Se puede hacer : df.iloc[1 + 10]
df
df['timeend'] = pd.to_datetime(df['timeend'])
df['timeend'] = df['timeend'].dt.date
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10334 entries, 0 to 10333
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   timeend   10334 non-null  object 
 1   valor     10334 non-null  float64
 2   seriesId  10334 non-null  int64  
 3   siteCode  10334 non-null  int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 323.1+ KB


# Limpiando los datos de los rios para estandarizar su altura segun el promedio de los registros

In [5]:
df_limpio = df.groupby(["timeend", "seriesId", "siteCode"])["valor"].mean().reset_index()

df_limpio['valor'] = df_limpio['valor'].round(2)
df_limpio

,timeend,seriesId,siteCode,valor
0,2025-09-18,8,8,0.66
1,2025-09-18,9,9,8.60
2,2025-09-18,10,10,9.20
3,2025-09-18,11,11,6.30
4,2025-09-18,12,12,6.62
...,...,...,...,...
784,2025-09-19,37580,7387,1.47
785,2025-09-19,37581,7388,0.59
786,2025-09-19,37594,7392,1.44
787,2025-09-19,37595,6624,0.63


In [6]:
df_limpio = df_limpio.sort_values(by='seriesId')
df_limpio

,timeend,seriesId,siteCode,valor
0,2025-09-18,8,8,0.66
393,2025-09-19,8,8,0.46
394,2025-09-19,9,9,8.40
1,2025-09-18,9,9,8.60
2,2025-09-18,10,10,9.20
...,...,...,...,...
390,2025-09-18,37594,7392,1.45
391,2025-09-18,37595,6624,0.63
787,2025-09-19,37595,6624,0.63
392,2025-09-18,37597,6622,1.12


In [7]:
df_limpio.to_csv('altura_rios_limpio.csv')

# Una vez generado el df 

In [26]:
df = pd.read_csv('altura_rios_limpio.csv')
df = df.drop(columns=['Unnamed: 0'])
df.to_csv('altura_rios_limpio.csv')
df['seriesId'] = df['seriesId'].astype(str)
df['siteCode'] = df['siteCode'].astype(str)
df['rio_id'] = df['seriesId']+"-"+df['siteCode']
df = df.drop('seriesId', axis=1)
df = df.drop('siteCode', axis=1)
df


,timeend,valor,rio_id
0,2025-09-18,0.66,8-8
1,2025-09-19,0.46,8-8
2,2025-09-19,8.40,9-9
3,2025-09-18,8.60,9-9
4,2025-09-18,9.20,10-10
...,...,...,...
784,2025-09-18,1.45,37594-7392
785,2025-09-18,0.63,37595-6624
786,2025-09-19,0.63,37595-6624
787,2025-09-18,1.12,37597-6622


In [25]:
# Uniendo lat y long con el json de leo
df_cord = pd.read_json('alturasHidrometricas.json')
df_cord = df_cord.drop(columns=['estacion_nombre', 'estacion_tabla','red_nombre','varid', 'var_nombre', 'procid', 'proc_nombre','unitid', 'unit_nombre', 'from_date', 'to_date', 'obs_count'])
df_cord['seriesid'] = df_cord['seriesid'].astype(str)
df_cord['sitecode'] = df_cord['sitecode'].astype(str)
df_cord['rio_id'] = df_cord['seriesid']+"-"+df_cord['sitecode']
df_cord.drop(columns=['seriesid', 'sitecode'])

,lat,lon,rio_id
0,-25.583333,-53.983333,8-8
1,-25.583333,-54.566667,9-9
2,-25.916679,-54.621134,10-10
3,-26.383333,-54.700000,11-11
4,-26.800000,-55.033333,12-12
...,...,...,...
415,-43.349722,-70.879722,37577-7385
416,-32.421944,-63.244722,37578-7386
417,-41.950556,-71.375278,37580-7387
418,-33.307222,-68.061389,37581-7388


In [29]:
df_cord_final = pd.merge(
    df,
    df_cord[['lat', 'lon', 'rio_id']],
    on= ['rio_id'],
    how= 'left'
)
df_cord_final.head()

,timeend,valor,rio_id,lat,lon
0,2025-09-18,0.66,8-8,-25.583333,-53.983333
1,2025-09-19,0.46,8-8,-25.583333,-53.983333
2,2025-09-19,8.40,9-9,-25.583333,-54.566667
3,2025-09-18,8.60,9-9,-25.583333,-54.566667
4,2025-09-18,9.20,10-10,-25.916679,-54.621134


In [ ]:
lluvias = pd.read_json('lluvias.json')
lluvias.head()
def crear_datos(seriesId, siteCode):
    #URL
    url = f'https://alerta.ina.gob.ar/pub/datos/datos&timeStart=2025-09-18&timeEnd=2025-09-19&seriesId={seriesId}&siteCode={siteCode}&varId=1&format=csv'
    

,seriesid,sitecode,estacion_nombre,estacion_tabla,red_nombre,varid,var_nombre,procid,proc_nombre,unitid,unit_nombre,from_date,to_date,obs_count,lat,lon
0,31971,2115,Paraná - Corrientes,sat2,RHN - SAT,1,precipitación diaria 12Z,1,medición directa,22,milímetros por día,2023-01-11T09:00:00,2025-09-01T09:00:00,866,-27.459722,-58.833333
1,32283,2122,Atuel - Paso Maroma,sat2,RHN - SAT,1,precipitación diaria 12Z,1,medición directa,22,milímetros por día,2023-01-11T09:00:00,2025-09-01T09:00:00,961,-36.842500,-66.995278
2,7013,2153,Lago Argentino - Calafate,sat2,RHN - SAT,1,precipitación diaria 12Z,1,medición directa,22,milímetros por día,2019-07-27T09:00:00,2025-09-01T09:00:00,1988,-50.320000,-72.261111
3,7014,2154,Bermejo - Pozo Sarmiento,sat2,RHN - SAT,1,precipitación diaria 12Z,1,medición directa,22,milímetros por día,2019-07-12T09:00:00,2025-08-31T09:00:00,2032,-23.140556,-64.196944
4,7015,2155,Neuquén - Chos Malal,sat2,RHN - SAT,1,precipitación diaria 12Z,1,medición directa,22,milímetros por día,2019-06-18T09:00:00,2025-09-01T09:00:00,2125,-37.375556,-70.270000


In [42]:
import pandas as pd
import requests
import json
from io import StringIO
from urllib.parse import urlencode

# --- CONFIGURACIÓN DE PARÁMETROS FIJOS ---
BASE_URL = "https://alerta.ina.gob.ar/pub/datos/datos"
TIME_START = "2023-01-01" 
TIME_END = "2023-01-03" 
JSON_FILE = 'lluvias.json'
OUTPUT_CSV = 'datos_precipitaciones_unificado.csv'

# Lista para almacenar los DataFrames de cada solicitud exitosa
todos_los_datos = []

# ====================================================================
# FUNCIÓN PRINCIPAL: SOLICITUD, LIMPIEZA Y TRANSFORMACIÓN
# ====================================================================
# ... (Configuración y librerías se mantienen igual) ...

# Modificamos la función para siempre devolver un DataFrame con metadatos
def obtener_datos_estacion(series_id, site_code, lat, lon):
    params = {
        'timeStart': TIME_START,
        'timeEnd': TIME_END,
        'seriesId': series_id,
        'siteCode': site_code,
        'varId': 1,
        'format': 'csv'
    }
    full_url = f"{BASE_URL}?{urlencode(params)}"
    
    # DataFrame vacío con las columnas finales esperadas para usar como fallback
    columnas_finales = ['obs_id', 'timestart', 'timeend', 'valor', 'timeupdate', 
                        'seriesId', 'siteCode', 'lat', 'lon']
    df_vacio = pd.DataFrame(columns=columnas_finales)
    
    try:
        response = requests.get(full_url, timeout=15)
        response.raise_for_status()

        contenido_texto = response.content.decode('latin-1') 
        lineas = contenido_texto.split('\n')
        start_index = -1
        
        # 1. Búsqueda de la Cabecera
        for i, line in enumerate(lineas):
            if line.strip().startswith('obs_id,timestart,timeend'):
                start_index = i
                break
        
        # 2. Si el Encabezado NO se encuentra (Datos ausentes)
        if start_index == -1:
            print(f"⚠️ Aviso: Estación {site_code} no devolvió datos de precipitación en {TIME_START} - {TIME_END}. Guardando metadatos.")
            # 💡 Retornamos el DataFrame vacío pero con las columnas necesarias
            return df_vacio 
        
        # 3. Si el Encabezado SÍ se encuentra (Procesamiento normal)
        datos_csv = '\n'.join(lineas[start_index:])
        csv_data = StringIO(datos_csv)
        df = pd.read_csv(csv_data, skipinitialspace=True)
        
        # Limpieza estándar
        df = df[~df['obs_id'].astype(str).str.startswith('###')].reset_index(drop=True)
        
        # Si el DF está vacío después de la limpieza, devolvemos el DF vacío
        if df.empty or 'obs_id' not in df.columns:
            print(f"⚠️ Aviso: Estación {site_code} devolvió un DF vacío después de la limpieza. Guardando metadatos.")
            return df_vacio
            
        # 4. Agregar Metadatos
        df['seriesId'] = series_id
        df['siteCode'] = site_code
        df['lat'] = lat
        df['lon'] = lon
        
        print(f"✅ Éxito: Datos obtenidos para Estación {site_code} (Filas: {len(df)})")
        return df

    except requests.exceptions.RequestException as e:
        print(f"❌ Error en la Solicitud para {site_code}: {e}")
    except Exception as e:
        print(f"❌ Error inesperado al procesar {site_code}: {e}")
    
    # En caso de error, aseguramos que el DataFrame vacío con las columnas se devuelve
    return df_vacio

# -------------------------------------------------------------
# 💡 IMPORTANTE: Debes asegurarte de que tu lista 'estaciones'
# en el script principal se esté iterando correctamente, y que
# el final del script (donde se unen y guardan los datos)
# use 'df_vacio' para rellenar los metadatos.
# -------------------------------------------------------------
# ====================================================================
# EJECUCIÓN DEL ETL
# ====================================================================
try:
    # 1. Carga del JSON (Extracción)
    with open(JSON_FILE, 'r', encoding='utf-8') as f:
        estaciones = json.load(f)
        
    print(f"Total de estaciones encontradas en '{JSON_FILE}': {len(estaciones)}")

    # 2. Iterar y Solicitar datos
    for estacion in estaciones:
        
        # Conversión a enteros y flotantes antes de pasar a la función
        series_id = int(estacion['seriesid'])
        site_code = int(estacion['sitecode'])
        lat = float(estacion['lat'])
        lon = float(estacion['lon'])
        
        # Llamada a la función
        df_estacion = obtener_datos_estacion(series_id, site_code, lat, lon)
        
        # Si la solicitud fue exitosa, guardamos el DataFrame
        if df_estacion is not None:
            todos_los_datos.append(df_estacion)

    # 3. Unificación y Carga Final (ETL)
    if todos_los_datos:
        # Concatenar todos los DataFrames en uno solo
        df_final = pd.concat(todos_los_datos, ignore_index=True)
        
        # Guardar en el archivo CSV
        df_final.to_csv(OUTPUT_CSV, index=False, encoding='utf-8')
        
        print(f"\n✨ Proceso Completado. Se guardó el archivo '{OUTPUT_CSV}' con un total de {len(df_final)} registros.")
    else:
        print("\n⚠️ No se obtuvieron datos de ninguna estación. Revisar el archivo JSON y la conexión a la API.")

except FileNotFoundError:
    print(f"FATAL ERROR: El archivo '{JSON_FILE}' no fue encontrado.")
except json.JSONDecodeError:
    print(f"FATAL ERROR: El archivo '{JSON_FILE}' no es un JSON válido.")
except Exception as e:
    print(f"FATAL ERROR GENERAL: {e}")

Total de estaciones encontradas en 'lluvias.json': 234
⚠️ Aviso: Estación 2115 no devolvió datos de precipitación en 2023-01-01 - 2023-01-03. Guardando metadatos.
⚠️ Aviso: Estación 2122 no devolvió datos de precipitación en 2023-01-01 - 2023-01-03. Guardando metadatos.
⚠️ Aviso: Estación 2153 no devolvió datos de precipitación en 2023-01-01 - 2023-01-03. Guardando metadatos.
⚠️ Aviso: Estación 2154 no devolvió datos de precipitación en 2023-01-01 - 2023-01-03. Guardando metadatos.
⚠️ Aviso: Estación 2155 no devolvió datos de precipitación en 2023-01-01 - 2023-01-03. Guardando metadatos.


KeyboardInterrupt: 